In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import astropy.io.ascii as ascii
import platedesign.observatory as observatory
import datetime
import PyAstronomy.pyasl as pyasl
from moonphase import moonphase2

## Initial set up

Using latest schedule; this is used to determine twilights.

In [2]:
rdr = ascii.get_reader(Reader=ascii.Basic)
schedule = rdr.read('Sch_base.Aug16.RM_ELG.txt')

Consider only dates in Years 4-6.

In [3]:
ischedule = np.where((schedule['MJD'] > 2400000.5 + 58000) &
                     (schedule['Eng'] == 0))[0]

The Observatory object contains some information about the observatory; here only longitude and latitude matter.

In [4]:
apo = observatory.Observatory('APO')

We calculate properties for every 5 minutes of observing time.

In [5]:
interval_hours = 5. / 60.
interval_days = interval_hours / 24.

In [6]:
ninterval = 0
exposures = np.zeros(0, dtype=np.float64)
for day in schedule[ischedule]:
    jd_start = 0.
    jd_end = 0.
    if(day['MJD_start_bright'] == 0.):
        jd_start = day['MJD_start_dark']
        jd_end = day['MJD_end_dark']
    elif(day['MJD_start_dark'] == 0.):
        jd_start = day['MJD_start_bright']
        jd_end = day['MJD_end_bright']
    else:
        jd_start = np.array([day['MJD_start_bright'], day['MJD_start_dark']]).min()
        jd_end = np.array([day['MJD_end_bright'], day['MJD_end_dark']]).max()
    #print("jd_start={jd_start:>0.5f} jd_end={jd_end:>0.5f}".format(jd_start=jd_start, jd_end=jd_end))
    ninterval_tmp = np.int32(np.floor((jd_end - jd_start) / interval_days))
    exposures_tmp = (np.arange(ninterval_tmp) + 0.5) * interval_days + jd_start
    exposures = np.append(exposures, exposures_tmp)
    ninterval = ninterval + ninterval_tmp

These routines use the basic Meeus-based calculations for moon position, LST, moon illumination, and fractional phase. 

In [7]:
moon_ras, moon_decs, dist, geolon, geolat = pyasl.moonpos(exposures)
lsts = 15. * pyasl.ct2lst(exposures, np.zeros(len(exposures)) + apo.location.longitude.value)
moon_has = lsts - moon_ras

In [8]:
moon_alts, moon_azs = pyasl.hadec2altaz(moon_has, moon_decs, np.zeros(len(exposures)) + apo.location.latitude.value)

In [9]:
moon_illuminations = moonphase2(exposures)
phases = 1. - np.arccos(1.99999 * (moon_illuminations - 0.5)) / np.pi

## Calculating grey time as defined for ELGs

We use the moon illumination to calculate the grey time as supposed to be currently defined for the ELGs (0.35 < illum. < 0.45, when altitude < 10.). This illumination is the same quantity calculated and used in [the master schedule software](https://trac.sdss.org/browser/repo/sdss/master_schedule/trunk/plan_night_sdss4cor_out.pro).

In [10]:
dark = np.zeros(24)
grey = np.zeros(24)
bright = np.zeros(24)
for (lst, moon_alt, moon_illumination, phase) in zip(lsts, moon_alts, moon_illuminations, phases):
    ilst = np.int32(np.floor(lst / 15.))
    if((moon_illumination < 0.35) or
       ((moon_alt < 0.) and (moon_illumination < 0.75))):
        dark[ilst] = dark[ilst] + interval_hours 
    else:
        if((moon_illumination > 0.35) and
           (moon_illumination < 0.45) and
           (moon_alt < 10.)):
            grey[ilst] = grey[ilst] + interval_hours
        else:
            bright[ilst] = bright[ilst] + interval_hours

In [11]:
print(dark.sum() / (dark.sum() + bright.sum() + grey.sum()))
print(grey.sum() / (dark.sum() + bright.sum() + grey.sum()))
print(bright.sum() / (dark.sum() + bright.sum() + grey.sum()))

0.536193950194
0.00556901400607
0.458237035799


Over three years, there should be only 80 hours total of such grey time.

In [12]:
print(dark.sum())
print(grey.sum())
print(bright.sum())

4814.08333333
50.0
4114.16666667


## Calculating proposed grey time

There's a claim that there is much more time if we let the Moon go to 35 deg altitude.

In [13]:
dark = np.zeros(24)
grey = np.zeros(24)
bright = np.zeros(24)
for (lst, moon_alt, moon_illumination, phase) in zip(lsts, moon_alts, moon_illuminations, phases):
    ilst = np.int32(np.floor(lst / 15.))
    if((moon_illumination < 0.35) or
       ((moon_alt < 0.) and (moon_illumination < 0.75))):
        dark[ilst] = dark[ilst] + interval_hours 
    else:
        if((moon_illumination > 0.35) and
           (moon_illumination < 0.45) and
           (moon_alt < 35.)):
            grey[ilst] = grey[ilst] + interval_hours
        else:
            bright[ilst] = bright[ilst] + interval_hours

This only adds 110 hours over three years (not 200 as eBOSS claims JUST for 10 < alt < 20!!).

In [14]:
print(dark.sum())
print(grey.sum())
print(bright.sum())

4814.08333333
190.166666667
3974.0


## Just dark and bright

This should be the standard dark/bright split

In [15]:
dark = np.zeros(24)
bright = np.zeros(24)
for (lst, moon_alt, moon_illumination, phase) in zip(lsts, moon_alts, moon_illuminations, phases):
    ilst = np.int32(np.floor(lst / 15.))
    if((moon_illumination < 0.35) or
       ((moon_alt < 0.) and (phase < 0.75))):
        dark[ilst] = dark[ilst] + interval_hours 
    else:
        bright[ilst] = bright[ilst] + interval_hours

In [16]:
print(dark.sum() / (dark.sum() + bright.sum()))

0.554729485145


In [17]:
print(dark.sum())
print(bright.sum())

4980.5
3997.75
